In [67]:
import pandas as pd

In [68]:

def get_only_uniprot_accession(df, colname):
    
    colname_out = colname + '_uniprot_accession'
    
    df[colname_out] = df[colname].str.split('-', expand=True)[1]
    
    return df


def calc_coverage(df, which_one ,start, end, lenght):
    
    col_name = 'COV_' + which_one
    
    df[col_name] = (df.loc[: , end] - df.loc[: , start]) / df.loc[: , lenght]
    
    df[col_name] = df[col_name].round(2)
    
    return df

In [69]:
df = pd.read_csv('../../../results/foldseek_search_TSV/cluster_representer_vs_Uniprot50.tsv', 
                 sep='\t',  
                 names=['query','target','alnlen','fident', 'evalue', 'qstart','qend', 'qlen','tstart','tend','tlen'])

#kinetoplastea annotation data
#one structure per row
df_kineto_annotation = pd.read_csv('../../protein_data_bases/annotation_info/kinetoplastea_taxid5653_annotation_info_from_uniprot.tsv', sep='\t', low_memory=False)


#estructure to genID to OG

df_OG_genID_uniprot  = pd.read_csv('../../report/ortholog_groups_x_sequence_clustering_x_UNIPROT.tsv', sep='\t')

#hypothetical OG data
df_hypothetical_OG = pd.read_csv('../../report/Ortholog_Group_Full_of_hypotetical_boolean.tsv', sep='\t')

In [70]:
df = (df
 .pipe(get_only_uniprot_accession, 'query')
 .pipe(get_only_uniprot_accession, 'target')
 .pipe(calc_coverage, 'query', 'qstart', 'qend','qlen')
 .pipe(calc_coverage, 'target', 'tstart', 'tend','tlen'))



In [71]:
df = df[df['query_uniprot_accession'] != df['target_uniprot_accession']]
df.shape

(4083529, 15)

In [72]:
#tengo que sacar los kinetoplastidos porque seguro tengo hit con ellos 

uniprot_kineto = df_kineto_annotation['Entry'].unique()
print('Dato curioso e importante')

cuantos = df[~df.query_uniprot_accession.isin(uniprot_kineto) ]['query'].nunique()
print(f'Hay {cuantos} proteinas en los query que no estan dentro de mi archivo de  prot de kinetoplastidos de uniprot')





Dato curioso e importante
Hay 55 proteinas en los query que no estan dentro de mi archivo de  prot de kinetoplastidos de uniprot


In [73]:
df.query_uniprot_accession.nunique()

10589

In [74]:
#esto no se si es necesario porque tengo que ver en el uniprot50 cuales son los genes que estan dentro de este grupo
df = df[~df.target_uniprot_accession.isin(uniprot_kineto) ]

In [75]:
df.query_uniprot_accession.nunique()

7229

In [76]:
df = df.sort_values(by=['fident', 'evalue'], ascending=[False, True]).drop_duplicates(subset=['query_uniprot_accession'], keep='first')

In [77]:
df = df.merge(df_OG_genID_uniprot, left_on='query_uniprot_accession', right_on='uniprot', how='left')

In [78]:
df = df.merge(df_hypothetical_OG, left_on='Ortholog_Group', right_on='cluster_representer_or_OG')

In [79]:
df_htpotetical = df[df.only_hypothetical_OG == True]

In [80]:
df_htpotetical.query_uniprot_accession.nunique()

495

In [81]:
df_htpotetical.sort_values('evalue').drop_duplicates(subset=['query_uniprot_accession'])#.head(20) [df_htpotetical['query_uniprot_accession'] != df_htpotetical['target_uniprot_accession'] ]

,query,target,alnlen,fident,evalue,qstart,qend,qlen,tstart,tend,...,COV_query,COV_target,Ortholog_Group,Gene ID,uniprot,pLDDT_mean,cluster_representer_or_OG,OGsize,OGhypotetical_proteins,only_hypothetical_OG
1,AF-A0A3Q8IQ36-F1-model_v4.pdb,AF-E9BJB4-F1-model_v4,2105,1.000,3.317000e-54,0,353,354,0,353,...,1.00,1.00,JKF63_03371,LdBPK_271840.1,A0A3Q8IQ36,68.0,JKF63_03371,31,31.0,True
1252,AF-A4H327-F1-model_v4.pdb,AF-E9B776-F1-model_v4,747,0.490,5.346000e-40,0,572,573,0,656,...,1.00,1.00,EMOLV88_010006000,LbrM.01.0150,A4H327,49.0,EMOLV88_010006000,33,33.0,True
4286,AF-A4HSG5-F1-model_v4.pdb,AF-A0A0L0DUZ2-F1-model_v4,874,0.252,3.092000e-28,7,649,675,2,465,...,0.95,0.99,CFAC1_020011100,LINF_050009300,A4HSG5,77.0,CFAC1_020011100,36,36.0,True
4889,AF-Q57VW3-F1-model_v4.pdb,AF-A0A814Q343-F1-model_v4,872,0.222,3.814000e-28,0,670,1403,0,495,...,0.48,0.96,TcIL3000_5_150,Tb427.05.500,Q57VW3,60.0,TcIL3000_5_150,10,10.0,True
5,AF-A0A504Y185-F1-model_v4.pdb,AF-E9BC56-F1-model_v4,517,0.994,1.062000e-22,0,181,182,0,181,...,0.99,0.99,LTULEM423_150006500,LdCL_150006400,A0A504Y185,48.0,LTULEM423_150006500,23,23.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4863,AF-A0A3P3ZAZ1-F1-model_v4.pdb,AF-A0A1S3BZJ2-F1-model_v4,190,0.224,9.737000e-06,0,162,163,13,139,...,0.99,0.90,PCON_0084390,LbrM.28.1040,A0A3P3ZAZ1,69.0,PCON_0084390,36,36.0,True
5495,AF-A4I845-F1-model_v4.pdb,AF-A0A5A8E8M6-F1-model_v4,79,0.196,9.741000e-06,483,1536,1554,1,1123,...,0.68,1.00,Lsey_0196_0080,LINF_320028500,A4I845,65.0,Lsey_0196_0080,15,15.0,True
5093,AF-Q57VQ5-F1-model_v4.pdb,AF-A0A7S1JCG2-F1-model_v4,196,0.214,9.819000e-06,0,182,463,157,325,...,0.39,0.51,Lsey_0220_0100,Tb927.7.590,Q57VQ5,76.0,Lsey_0220_0100,63,63.0,True
6455,AF-A4HZB9-F1-model_v4.pdb,AF-A0A7S1F4G6-F1-model_v4,87,0.156,9.913000e-06,674,1363,1372,175,858,...,0.50,0.79,LmjF.21.1220,LINF_210020300,A4HZB9,64.0,LmjF.21.1220,34,34.0,True


In [16]:
df_OG_genID_uniprot[df_OG_genID_uniprot.Ortholog_Group == 'CFAC1_040020800']

,Ortholog_Group,Gene ID,uniprot,pLDDT_mean
3260,CFAC1_040020800,Tb927.8.4660,Q57UY5,84.0


In [17]:
!grep "CFAC1_040020800" ../../../config/mandatory_files/Ortholog_group_to_geneID.tsv

CFAC1_040020800	CFAC1_040020800
CFAC1_040020800	Lsey_0109_0140
CFAC1_040020800	LpyrH10_22_1630
CFAC1_040020800	LGELEM452_100017900
CFAC1_040020800	LmxM.10.1200
CFAC1_040020800	LINF_100019400
CFAC1_040020800	LTRL590_100013800
CFAC1_040020800	LbrM.10.1320
CFAC1_040020800	GH5_06993
CFAC1_040020800	LtaP10.1600
CFAC1_040020800	LDHU3_10.1660
CFAC1_040020800	LMJSD75_100019500
CFAC1_040020800	LMJFC_100020200
CFAC1_040020800	LSCM4_06670
CFAC1_040020800	LAMA_000168600
CFAC1_040020800	LdCL_100020000
CFAC1_040020800	LENLEM3045_100018100
CFAC1_040020800	LARLEM1108_100017700
CFAC1_040020800	JKF63_07681
CFAC1_040020800	LdBPK_101300.1
CFAC1_040020800	EMOLV88_100013000
CFAC1_040020800	LbrM.10.2.001320
CFAC1_040020800	LdBPK.10.2.001300
CFAC1_040020800	CUR178_07414
CFAC1_040020800	LMARLEM2494_100017100
CFAC1_040020800	LPAL13_100018300
CFAC1_040020800	LMJLV39_100019500
CFAC1_040020800	LAEL147_000144300
CFAC1_040020800	LSCM1_07122
CFAC1_040020800	LBRM2903_100022500
CFAC1_040020800	LmjF.10.1200
CFAC1_040020

In [18]:
!grep "LpyrH10_02_5090" ../../../config/mandatory_files/Ortholog_group_to_geneID.tsv

TcIL3000.11.12350	LpyrH10_02_5090
